In [ ]:
%matplotlib inline
import pylab as plt
from astrometry.util.fits import fits_table
from astrometry.util.util import Tan
from astrometry.util.starutil_numpy import degrees_between
import fitsio
import numpy as np
import os
from collections import Counter
from desimodel.io import load_focalplane

In [ ]:
(fp, excl, pos, time) = load_focalplane()

In [ ]:
time

In [ ]:
pos[:10]

In [ ]:
state = pos['STATE'].data
#excl = pos['EXCLUSION'].data
loc = pos['LOCATION'].data

In [ ]:
Counter(state)

In [ ]:
for i in range(32):
    bitval = 1<<i
    if np.any(state & bitval > 0):
        print('bit', i+1, '(0x%x)'%bitval, 'set for', np.sum(state & bitval > 0), 'positioners')

In [ ]:
np.sum((state & 0x2) > 0)

In [ ]:
np.sum((state & 0x4) > 0)

In [ ]:
np.sum((state & 0x6) > 0)

In [ ]:
sum(state != 0)

In [ ]:
Ibad = np.flatnonzero(state)

In [ ]:
pos[Ibad]

In [ ]:
bad_loc = loc[Ibad]

In [ ]:
os.environ['DESI_SPECTRO_DATA']

In [ ]:
fn1 = 'desi/spectro/data/20210405/00083539/coordinates-00083539.fits'
T = fits_table(fn1)
hdr1 = fitsio.read_header(fn1)
tilera1, tiledec1 = hdr1['TILERA'], hdr1['TILEDEC']
fieldrot1 = hdr1['FIELDROT']

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(T.target_ra, T.target_dec, 'b.');
ax = plt.axis()
plt.plot(T.fiber_ra, T.fiber_dec, 'r.')
plt.axis(ax);

In [ ]:
plt.figure(figsize=(10,10))
plt.quiver(T.target_ra, T.target_dec, T.fiber_ra-T.target_ra, T.fiber_dec-T.target_dec);


In [ ]:
D = np.array([degrees_between(r1,d1,r2,d2) for r1,d1,r2,d2 in zip(T.target_ra, T.target_dec, T.fiber_ra, T.fiber_dec)])
plt.scatter(T.target_ra, T.target_dec, c=D, s=5, vmin=0, vmax=0.001)
plt.colorbar();

In [ ]:
plt.hist(D*3600, log=True, bins=50, range=(0,5));

In [ ]:
Ibad = np.flatnonzero((D*3600.)>1)

In [ ]:
len(Ibad)

In [ ]:
plt.figure(figsize=(8,8))
plt.plot(T.target_ra, T.target_dec, 'k.', alpha=0.1)
plt.plot(T.target_ra[Ibad], T.target_dec[Ibad], 'r.')
ax = plt.axis()
plt.axis([ax[1],ax[0], ax[2],ax[3]]);

In [ ]:
T.about()

In [ ]:
plt.plot(T.fiber_x[Ibad], T.fiber_y[Ibad], 'k.')
plt.axis('equal');

In [ ]:
fn2 = 'desi/spectro/data/20210405/00083537/coordinates-00083537.fits'
T2 = fits_table(fn2)
hdr2 = fitsio.read_header(fn2)
tilera2,tiledec2 = hdr2['TILERA'], hdr2['TILEDEC']
fieldrot2 = hdr2['FIELDROT']

In [ ]:
T.fid = T.petal_loc*1000 + T.device_loc
T2.fid = T2.petal_loc*1000 + T2.device_loc

In [ ]:
np.all(T.fid == T2.fid)

In [ ]:
plt.figure(figsize=(10,10))
plt.quiver(T2.fiber_x, T2.fiber_y, T2.fiber_x-T.fiber_x, T2.fiber_y-T2.fiber_y);
dxy = np.hypot(T2.fiber_x-T.fiber_x, T2.fiber_y-T2.fiber_y)

In [ ]:
plt.hist(dxy, log=True, range=(0,0.1), bins=50);

In [ ]:
Ibad2 = np.flatnonzero(dxy < 0.1)

In [ ]:
len(Ibad2)

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(T2.fiber_x, T2.fiber_y, 'k.', alpha=0.1)
plt.plot(T2.fiber_x[Ibad2], T2.fiber_y[Ibad2], 'r.');

In [ ]:
### Check whether going fiber_{ra,dec} -> fake WCS x,y -> fiber_{ra,dec} works (at different Decs/airmasses)
# (for the bad fibers only)

In [ ]:
def rotate_by(x, y, fieldrot):
    c,s = np.cos(np.deg2rad(fieldrot)), np.sin(np.deg2rad(fieldrot))
    R = np.array([[c, -s], [s, c]])
    rx,ry = np.dot(R, np.vstack((x,y)))
    return rx,ry

In [ ]:
cd = 1./3600.
fakewcs1 = Tan(tilera1, tiledec1, 0., 0., -cd, 0., 0., cd, 12000., 12000.)
ok,x1,y1 = fakewcs1.radec2pixelxy(T.fiber_ra, T.fiber_dec)
Igood = np.flatnonzero(T.fiber_x)
rx1,ry1 = rotate_by(x1, y1, +fieldrot1)
x1,y1 = rx1,ry1
Counter(ok)

In [ ]:
fakewcs2 = Tan(tilera2, tiledec2, 0., 0., -cd, 0., 0., cd, 12000., 12000.)

In [ ]:
x2,y2 = rotate_by(x1,y1, -fieldrot2)
rr,dd = fakewcs2.pixelxy2radec(x2, y2)

In [ ]:
D = np.array([degrees_between(r1,d1,r2,d2) for r1,d1,r2,d2 in zip(rr, dd, T2.fiber_ra, T2.fiber_dec)])
plt.hist(3600.*D[Igood], log=True, range=(0, 10), bins=25);

In [ ]:
plt.figure(figsize=(8,8))
Istuck = Igood[(D[Igood] < 2./3600.)]
plt.quiver(rr[Istuck], dd[Istuck], (rr - T2.fiber_ra)[Istuck], (dd - T2.fiber_dec)[Istuck]);
plt.axis('equal')

In [ ]:
from glob import glob
from functools import reduce

In [ ]:
len(bad_loc)

In [ ]:
fns = glob('desi/spectro/data/20210405/*/coordinates-*.fits')
results = []
loc = 1000 * T.petal_loc + T.device_loc
is_stuck = np.isin(loc, bad_loc)
for fn3 in fns:
    try:
        T3 = fits_table(fn3)
        hdr3 = fitsio.read_header(fn3)
        tilera3,tiledec3 = hdr3['TILERA'], hdr3['TILEDEC']
        fieldrot3 = hdr3['FIELDROT']
        T3.fiber_ra
    except:
        continue
    #T.petal_loc*1000 + T.device_loc
    # Drop "stuck" ones that are missing data in this frame.
    Istuck = is_stuck * (T3.fiber_x != 0.) * np.isfinite(T3.fiber_ra)
    
    fakewcs3 = Tan(tilera3, tiledec3, 0., 0., -cd, 0., 0., cd, 12000., 12000.)
    x3,y3 = rotate_by(x1,y1, -fieldrot3)
    rr,dd = fakewcs3.pixelxy2radec(x3, y3)
    D = np.array([degrees_between(r1,d1,r2,d2) for r1,d1,r2,d2 in zip(rr, dd, T3.fiber_ra, T3.fiber_dec)])
    results.append((rr, dd, T3.fiber_ra, T3.fiber_dec, (T3.fiber_x != 0.) * np.isfinite(T3.fiber_ra), 3600.*D))


In [ ]:
np.sum(is_stuck)

In [ ]:
allD = np.vstack([r[5] for r in results]).T
is_stuck = (np.mean(allD, axis=1) < 1)
plt.hist(np.mean(allD, axis=1), log=True, range=(0,10), bins=20);
np.sum(is_stuck)

In [ ]:
for (rr, dd, fiber_ra, fiber_dec, ok, D) in results:
    plt.figure(figsize=(12,6))
    plt.subplot(1,2,1)
    plt.hist(D[ok * is_stuck], log=True, bins=20, range=(0,5));
    plt.subplot(1,2,2)
    show = ok * is_stuck
    plt.quiver(rr[show], dd[show], (rr - fiber_ra)[show], (dd - fiber_dec)[show]);
    #plt.axis('equal')
    plt.show()
    break

In [ ]:
from desimodel.focalplane.fieldrot import field_rotation_angle
import astropy.time

In [ ]:
obsdate = astropy.time.Time('2022-07-01')
mjd = obsdate.mjd

In [ ]:
tilera1, tiledec1

In [ ]:
field_rotation_angle(tilera1, tiledec1, mjd)

In [ ]:
fieldrot1

In [ ]:
tiles = fits_table('/global/cfs/cdirs/desi/users/djschleg/tiling/tiles-sv3-rosette.fits')

In [ ]:
tiles.about()

In [ ]:
tiles.fieldrot = np.array([field_rotation_angle(ra, dec, mjd) for ra,dec in zip(tiles.ra, tiles.dec)])

In [ ]:
plt.hist(tiles.fieldrot);

In [ ]:
from astrometry.libkd.spherematch import tree_build_radec, tree_search_radec, tree_open

In [ ]:
skybrick_dir = '/global/cscratch1/sd/dstn/skybricks'
skytiles = fits_table(os.path.join(skybrick_dir, 'skybricks-exist.fits'))
skytiles_kd = tree_build_radec(ra=skytiles.ra, dec=skytiles.dec)

In [ ]:
skytiles_kd_fn = 'skytiles-kd.fits'
skytiles_kd.write(skytiles_kd_fn)

In [ ]:
def one_tile(X):
    tile, itile, x_stuck, y_stuck, skytiles, skytiles_kd_fn, skybrick_dir = X
    #print('Tile', itile)
    # ugh
    #skytiles_kd = tree_build_radec(ra=skytiles.ra, dec=skytiles.dec)
    skytiles_kd = tree_open(skytiles_kd_fn)
    fakewcs = Tan(tile.ra, tile.dec, 0., 0., -cd, 0., 0., cd, 12000., 12000.)
    x,y = rotate_by(x_stuck, y_stuck, -tile.fieldrot)
    rr,dd = fakewcs.pixelxy2radec(x, y)

    skyvals = np.empty(len(rr), np.int16)
    skyvals[:] = -1
    skyvals_margin = np.empty(len(rr), np.float32)
    skyvals_margin[:] = -1.

    margin = 2
    
    I = tree_search_radec(skytiles_kd, tile.ra, tile.dec, 2.5)
    for i in I:
        keep = np.flatnonzero(# unique skybrick area
                              (rr >= skytiles.ra1[i]) * (rr < skytiles.ra2[i]) *
                              (dd >= skytiles.dec1[i]) * (dd < skytiles.dec2[i]))
        if len(keep) == 0:
            continue

        sky,hdr = fitsio.read(os.path.join(skybrick_dir, 'sky-%s.fits.gz' % skytiles.brickname[i]), header=True)
        skywcs = Tan(hdr)
        h,w = sky.shape
        ok,xx,yy = skywcs.radec2pixelxy(rr, dd)
        xx -= 1.
        yy -= 1.
        ix = np.round(xx).astype(np.int32)
        iy = np.round(yy).astype(np.int32)
        keep = np.flatnonzero((ix >= 0) * (ix < w) * (iy >=0) * (iy < h) * 
                              # unique skybrick area
                              (rr >= skytiles.ra1[i]) * (rr < skytiles.ra2[i]) *
                              (dd >= skytiles.dec1[i]) * (dd < skytiles.dec2[i]))
        if len(keep) == 0:
            continue
        skyvals[keep] = sky[iy[keep], ix[keep]]

        for k in zip(keep):
            xlo,xhi = np.maximum(ix[k] - margin, 0), np.minimum(ix[k] + margin + 1, w)
            ylo,yhi = np.maximum(iy[k] - margin, 0), np.minimum(iy[k] + margin + 1, h)
            xg,yg = np.meshgrid(np.arange(xlo,xhi), np.arange(ylo,yhi))
            inrad = np.hypot(xx[k] - xg, yy[k] - yg) <= margin
            skyvals_margin[k] = np.mean(sky[ylo:yhi, xlo:xhi][inrad])
    return skyvals, skyvals_margin

In [ ]:
from astrometry.util.multiproc import multiproc

In [ ]:
x_stuck = x1[is_stuck]
y_stuck = y1[is_stuck]

tiles.stuck_skyvals = np.empty((len(tiles), len(x_stuck)), np.int16)
tiles.stuck_skyvals[:,:] = -1
tiles.stuck_skyvals_margin = np.empty((len(tiles), len(x_stuck)), np.float32)
tiles.stuck_skyvals_margin[:,:] = -1

mp = multiproc(8)
R = mp.map(one_tile, [(tile, itile, x_stuck, y_stuck, skytiles, skytiles_kd_fn, skybrick_dir) for itile,tile in enumerate(tiles)])

for i,(s,sm) in enumerate(R):
    tiles.stuck_skyvals[i,:] = s
    tiles.stuck_skyvals_margin[i,:] = sm


In [ ]:
tiles.writeto('tiles-sv3-stuck-sky.fits')

In [ ]:
tiles.stuck_skyvals.shape

In [ ]:
len(tiles), len(x_stuck)

In [ ]:
tiles.n_stuck_on_sky = np.sum(tiles.stuck_skyvals == 0, axis=1)
tiles.n_stuck_on_sky_margin = np.sum(tiles.stuck_skyvals_margin == 0, axis=1)

In [ ]:
plt.figure(figsize=(10,8))
plt.hist(tiles.n_stuck_on_sky / len(x_stuck), label='Nearest pixel');
plt.hist(tiles.n_stuck_on_sky_margin / len(x_stuck), histtype='step', lw=3, label='2" margin');
plt.xlabel('Fraction of stuck positioners that land on good sky')
plt.legend()
plt.ylabel('Number of tiles')
plt.title('SV3 rosettes: fraction of stuck positioners usable as sky fibers');
plt.savefig('sv3-sky.png')

In [ ]:
x_stuck = x1[is_stuck]
y_stuck = y1[is_stuck]

tiles.stuck_skyvals = np.empty((len(tiles), len(x_stuck)), np.int16)
tiles.stuck_skyvals[:,:] = -1
tiles.stuck_skyvals_margin = np.empty((len(tiles), len(x_stuck)), np.float32)
tiles.stuck_skyvals_margin[:,:] = -1

for itile,tile in enumerate(tiles):
    print('tile', itile, 'of', len(tiles))
    fakewcs = Tan(tile.ra, tile.dec, 0., 0., -cd, 0., 0., cd, 12000., 12000.)
    x,y = rotate_by(x_stuck, y_stuck, -tile.fieldrot)
    rr,dd = fakewcs.pixelxy2radec(x, y)

    skyvals = np.empty(len(rr), np.int16)
    skyvals[:] = -1
    skyvals_margin = np.empty(len(rr), np.float32)
    skyvals_margin[:] = -1.

    margin = 2
    
    I = tree_search_radec(skytiles_kd, tile.ra, tile.dec, 2.5)
    for i in I:
        sky,hdr = fitsio.read(os.path.join(skybrick_dir, 'sky-%s.fits.gz' % skytiles.brickname[i]), header=True)
        skywcs = Tan(hdr)
        h,w = sky.shape
        ok,xx,yy = skywcs.radec2pixelxy(rr, dd)
        xx -= 1.
        yy -= 1.
        ix = np.round(xx).astype(np.int32)
        iy = np.round(yy).astype(np.int32)
        keep = np.flatnonzero((ix >= 0) * (ix < w) * (iy >=0) * (iy < h) * 
                              # unique skybrick area
                              (rr >= skytiles.ra1[i]) * (rr < skytiles.ra2[i]) *
                              (dd >= skytiles.dec1[i]) * (dd < skytiles.dec2[i]))
        if len(keep) == 0:
            continue
        skyvals[keep] = sky[iy[keep], ix[keep]]

        for k in zip(keep):
            xlo,xhi = np.maximum(ix[k] - margin, 0), np.minimum(ix[k] + margin + 1, w)
            ylo,yhi = np.maximum(iy[k] - margin, 0), np.minimum(iy[k] + margin + 1, h)
            xg,yg = np.meshgrid(np.arange(xlo,xhi), np.arange(ylo,yhi))
            inrad = np.hypot(xx[k] - xg, yy[k] - yg) <= margin
            skyvals_margin[k] = np.mean(sky[ylo:yhi, xlo:xhi][inrad])

    tiles.stuck_skyvals[itile,:] = skyvals
    tiles.stuck_skyvals_margin[itile,:] = skyvals_margin
    #plt.hist(skyvals, bins=17)
    #plt.hist(skyvals_margin, bins=17, histtype='step')
    #plt.show()
    #break

In [ ]:
np.minimum(iy[keep]+margin+1, h)